In [1]:
import os
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import random
import implicit 
import matplotlib.pyplot as plt
import datetime
import random

os.environ['MKL_NUM_THREADS'] = '1'

In [2]:
data_train = pd.read_csv('rand_train_30.csv')
data_test = pd.read_csv('rand_test_30.csv')


data_test = data_test.sort_values(by=['userId','timestamp'])
data_test = data_test[['userId', 'movieId']].groupby(by=['userId']).agg(lambda x:list(x))

data_test = data_test.movieId

# Бейзлайн

Построим бейзлайн-модель самым простым образом, будем рекомендовать каждому пользователю топ 20 самых популярных фильмов, которые он еще не видел

In [3]:
def baseline_method(data_train, data_test):
    
    data_train = data_train.sort_values(by=['movieId'])
    mean_ratings = pd.DataFrame(list(data_train[['movieId', 'rating']].groupby(by=['movieId']).agg('mean').rating))
    mean_ratings['movieId'] =  data_train['movieId'].sort_values(ascending=False)
    mean_ratings['views'] = list(data_train.groupby(by=['movieId']).size())

    mean_ratings = mean_ratings.rename(columns={0:'rating'})
    mean_ratings = mean_ratings.sort_values(by=['views','rating'], ascending=[False, False])
    uniq_users = sorted(list(set(data_train['userId'])))
    
    cnt_uniq_users = len(uniq_users)
    
    
    list_recommendations = []
    i = 0
    for user_n in uniq_users:
        if (i % (cnt_uniq_users // 10) == 0) and (i != 0):
            print(f'Рекомендации готовы для {round((i / cnt_uniq_users) * 100)}% пользователей')
        movies_to_recommend = list(mean_ratings[mean_ratings.movieId.isin(data_train[data_train.userId == user_n]['movieId'].unique()) == False][0:20].movieId)
        list_recommendations.append(tuple([user_n, movies_to_recommend]))
        i += 1
        
    return list_recommendations

In [4]:
%%time
baseline_recommendations = baseline_method(data_train, data_test)

#бейзлайн рекомендации записываются в отдельный файл, чтобы повторно не запускать функцию
recs = pd.DataFrame()
recs['userId'] = uniq_users = sorted(list(data_train.userId.unique()))
list_recommends = [x[1] for x in baseline_recommendations]
recs['recommendations'] = list_recommends
recs.to_csv('baseline_recommendations_30.csv',sep=',', index=False)

Рекомендации готовы для 10% пользователей
Рекомендации готовы для 20% пользователей
Рекомендации готовы для 30% пользователей
Рекомендации готовы для 40% пользователей
Рекомендации готовы для 50% пользователей
Рекомендации готовы для 60% пользователей
Рекомендации готовы для 70% пользователей
Рекомендации готовы для 80% пользователей
Рекомендации готовы для 90% пользователей
Рекомендации готовы для 100% пользователей
Wall time: 13min 15s


## Метрика nDCG@20 для бейзлайн рекомендаций

In [5]:
## функции для подсчета метрики
def dcg(y_relevance: np.ndarray) -> float:
    return np.sum([(2**i - 1) / np.log2(k + 1) for (k, i) in enumerate(y_relevance, start=1)])

def ndcg(y_relevance: np.ndarray, k: int) -> float:
    
    if y_relevance.sum() == 0:
        return 0.0
    
    DCG = dcg(y_relevance[:k])
    IDCG = dcg(-np.sort(-y_relevance)[:k])
    return DCG / IDCG


def calc_ndcg(data_test, list_recommends, k=20):
    
    full_dcg_list = []
    j = 0
    for user, recom in list_recommends:

        dcg_list = []
        for rec in recom:
            if rec in data_test.loc[user]:
                dcg_list.append(1)
            else:
                dcg_list.append(0)
        full_dcg_list.append(dcg_list)
    
            
    ndcgs_list = []
    for nd in full_dcg_list:
        ndcgs_list.append(ndcg(np.array(nd), k))
    mean_ndcgs = np.mean(ndcgs_list)
    
    return mean_ndcgs

In [6]:
%%time
list_recommends = pd.read_csv('baseline_recommendations_30.csv', converters={'recommendations': pd.eval})
list_recommends = list(zip(list(list_recommends.userId),list(list_recommends.recommendations)))
calc_ndcg(data_test, list_recommends, k=20)

Wall time: 1min 5s


0.13116933649585474

Метрика nDCG@20 для бейзлайна на тесте составила 0.131

# Implicit ALS

Для подбора гиперпараметров используется следующая стретегия кросс-валидации: для каждого набора признаков модели из data_train выбираются по 5 рандомных фильмов для каждого пользователя, они отправляются во внутренний тест, на остальных данных модель обучается. Хоть мне и удалось реализовать функцию подсчета метрики, но подсчет метрики для каждого из пользователей занимает время, поэтому берется процент от всех пользователей, эти пользователи случайным образом выбираются и для них уже считается метрика, как показали проверки, разница между такой оценкой метрики и реальным значением присутствует лишь в третьем знаке после запятой.
P.S. Используется 2 фолда, то есть два случайных разбиения.

In [7]:
def user_movies_to_dict(data_train, data_test):
    '''
    Функция для созданий словаря пользователей
    '''
    
    user_dict = {}
    movie_dict = {}
    i = 0
    users = sorted(list(set(data_train['userId'].values)))
    for i in range(len(users)):
        user_dict[i] = users[i]
        i += 1
        
    return user_dict


random.seed(10)
def train_ALS(data_train, factor, reg, iters, user_dict,  rand_state=0):
    
    '''
    Функция которая обучает модель implicitALS,
    в соответсвии с заданными параметрами
    Возвращает обученную модель и разреженную матрицу
    sparse_user_item, которая используется для построения 
    рекомендаций
    
    '''
    
    sparse_user_item = sparse.csr_matrix((data_train['rating'].astype(float),
                                          (data_train['userId'],
                                           data_train['movieId'])))

    sparse_item_user = sparse.csr_matrix((data_train['rating'].astype(float),
                                          (data_train['movieId'],
                                           data_train['userId'])))
    
    model = implicit.als.AlternatingLeastSquares(factors=factor, regularization=reg, iterations=iters,
                                             num_threads=4, random_state=1)
    alpha_val = 40
    data_conf = (sparse_item_user * alpha_val).astype('double')
    model.fit(data_conf)
    
    
    
    return model, sparse_user_item

def test_score(model, data_train, data_test, sparse_user_item, percent_to_test=10):
  
    
    list_recommends = []
    
    list_users = list(data_test.index)
    len_users = len(list_users)
    
    for i in range(0, round((len(data_test) / 100) * percent_to_test)):  
        #юзеров для оценки каждый раз выбираем рандомно
        if percent_to_test < 100:
            new_user = random.choice(list_users)
        else:
            new_user = list_users[i]
        
        recommended = model.recommend(new_user, sparse_user_item, filter_already_liked_items=True, N=20)
        
        recs = list(map(lambda x:x[0], recommended))
        list_recommends.append(tuple([new_user, recs]))
    
        if (i % (len_users // 10) == 0) and (i != 0):
            print(f'Рекомендации для {round((i / len_users) * 100)} % пользователей посчитаны')
        
    ndcg_score = calc_ndcg(data_test, list_recommends, k=20)
    
    return ndcg_score

def tuning_model(full_data, factor, reg, iters, percent_to_test, data_test, rand_state=0):

    
    data_test = full_data.groupby(by='userId').sample(n=20, random_state=rand_state)
    
    data_train = full_data.merge(data_test, how='left', on=['timestamp', 'userId','movieId'])
    data_train = data_train[(data_train.rating_x > 0) 
                            & ((data_train.rating_y > 0)==False)].drop(['rating_y'], axis=1).rename(columns={'rating_x':'rating'})
    
    data_test = data_test.sort_values(by=['userId','timestamp'])
    data_test = data_test[['userId', 'movieId']].groupby(by=['userId']).agg(lambda x:list(x))

    data_test = data_test.movieId
    
    model, sparse_user_item = train_ALS(data_train, factor, reg, iters, percent_to_test)
    
    
    ndcg_score = test_score(model, data_train, data_test, sparse_user_item, percent_to_test=percent_to_test)

    return model, ndcg_score


def cross_val_score(iterations_list, regularizations_list, factors_list, percent_to_test, data_test, folds=2):
    random_state = 1

    dict_score = {}
    
    for iters in iterations_list:
        dict_score[iters] = {}
    
        for reg in regularizations_list:
            dict_score[iters][reg] = {}
        
            for factors in factors_list:
                dict_score[iters][reg][factors] = 0
                score_list = []
            
                for ii in range(folds):
                    now = datetime.datetime.now()
                    print(f'{now.hour}:{now.minute}:{now.second} factors: {factors}, regularization: {reg}, Iterations: {iters}  [{ii + 1}/{folds}]')
                    nd_score = tuning_model(data_train, factors, reg, iters, percent_to_test, data_test=data_test, rand_state=ii)[1]
                    print(f'nDCG@20 = {nd_score}')
                                            
                    score_list.append(nd_score)
                    
                    random_state += 1
                
                dict_score[iters][reg][factors] = np.mean(score_list)
                print(f'Среднее nDCG@20 по фолдам = {np.mean(score_list)}')
                
    return dict_score

In [8]:
%%time
#словари для юзеров и фильмов
user_dict = user_movies_to_dict(data_train, data_test)

Wall time: 1.3 s


In [9]:
%%time
iterations_list = [20, 40]
regularizations_list = [0.5, 0.7]
factors_list = [40, 70]
dict_score = cross_val_score(iterations_list, regularizations_list, factors_list, percent_to_test=5, data_test=data_test, folds=2)

18:56:23 factors: 40, regularization: 0.5, Iterations: 20  [1/2]


  0%|          | 0/20 [00:00<?, ?it/s]

nDCG@20 = 0.3325598101231601
18:58:42 factors: 40, regularization: 0.5, Iterations: 20  [2/2]


  0%|          | 0/20 [00:00<?, ?it/s]

nDCG@20 = 0.32910620454594697
Среднее nDCG@20 по фолдам = 0.33083300733455356
19:1:3 factors: 70, regularization: 0.5, Iterations: 20  [1/2]


...

nDCG@20 = 0.32293309183205926
Среднее nDCG@20 по фолдам = 0.32182920266644766
19:43:29 factors: 70, regularization: 0.7, Iterations: 40  [1/2]


  0%|          | 0/40 [00:00<?, ?it/s]

nDCG@20 = 0.32423075309022525
19:48:51 factors: 70, regularization: 0.7, Iterations: 40  [2/2]


  0%|          | 0/40 [00:00<?, ?it/s]

nDCG@20 = 0.3151240993914991
Среднее nDCG@20 по фолдам = 0.31967742624086215
Wall time: 57min 51s


In [12]:
%%time
model, sparse_user_item = train_ALS(data_train,
                              factor=40, reg=0.7,
                              iters=20,
                              user_dict=user_dict,
                              rand_state=0)

  0%|          | 0/20 [00:00<?, ?it/s]

Wall time: 1min 49s


In [13]:
%%time
ndcgs_score = test_score(model, data_train, data_test, sparse_user_item, percent_to_test=100)

Рекомендации для 10 % пользователей посчитаны
Рекомендации для 20 % пользователей посчитаны
Рекомендации для 30 % пользователей посчитаны
Рекомендации для 40 % пользователей посчитаны
Рекомендации для 50 % пользователей посчитаны
Рекомендации для 60 % пользователей посчитаны
Рекомендации для 70 % пользователей посчитаны
Рекомендации для 80 % пользователей посчитаны
Рекомендации для 90 % пользователей посчитаны
Рекомендации для 100 % пользователей посчитаны
Wall time: 2min 51s


In [14]:
print(f'Среднее значение метрики nDCG@20 на тесте для всех пользователей: {ndcgs_score}')

Среднее значение метрики nDCG@20 на тесте для всех пользователей: 0.42311919516785784


Метрика nDCG@20 на тестовой выборке составила 0.423